In [1]:
import pandas as pd
import numpy as np

In [2]:
y = pd.read_csv("cls_labelWithfea.csv")
print(len(y.loc[y["label"] == 0]))
print(len(y.loc[y["label"] == 1]))
y = y["label"]

32827
2418


In [3]:
fea_concat = pd.read_csv("sym_mesh_concat.csv",index_col=0)
print(fea_concat.shape)
fea_concat.head()

(35245, 692)


,0,1,2,3,4,5,6,7,8,9,...,38.1.1,39.1.1,40.1.1,41.1.1,42.1.1,43.1.1,44.1.1,45.1.1,46.1.1,47.1.1
0,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

from sklearn.utils import shuffle

from imblearn.under_sampling import RandomUnderSampler # Random UnderSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve,auc

## 负采样+划分测试集

In [5]:
import random as python_random
seed = 1
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.

rus = RandomUnderSampler(sampling_strategy=1,random_state=seed) 
rus_x, rus_y = rus.fit_resample(fea_concat ,y) #rus_y(Series)
print('负采样后数据集大小:',len(rus_y))

fea_train,fea_test,y_train,y_test=train_test_split(
                rus_x,
                rus_y,
                test_size=0.2, shuffle=True)

print(fea_train.shape,fea_test.shape)

负采样后数据集大小: 4836
(3868, 692) (968, 692)


## SVC

### GridSearchCV

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf=SVC(random_state=seed)

kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)

# kernels=["linear","poly", "rbf", "sigmoid"]
kernels=["rbf"]
gammas=["scale",0.000001,0.0000001,0.00000001]
# gammas=["scale",0.0001,0.00001,0.000001,0.0000001]
Cs=[1000,2000]
# Cs=[0.1,1,10,100,1000]
param_grid=dict(kernel=kernels,gamma=gammas,C=Cs)

scorings=["accuracy","roc_auc"]

grid = GridSearchCV(clf, param_grid, cv=kfold, 
                    scoring=scorings,return_train_score = True,refit="roc_auc",
                   verbose=2)
grid_result=grid.fit(fea_train,y_train)

#GridSearchCV results
params = grid_result.cv_results_["params"]
mean_train_accuracy=grid_result.cv_results_["mean_train_accuracy"]
mean_train_roc_auc=grid_result.cv_results_["mean_train_roc_auc"]

mean_test_accuracy=grid_result.cv_results_["mean_test_accuracy"]
mean_test_roc_auc=grid_result.cv_results_["mean_test_roc_auc"]

print("train_acc:",mean_train_accuracy)
print("train_auc:",mean_train_roc_auc)
print("test_acc:",mean_test_accuracy)
print("test_auc:",mean_test_roc_auc)

for mean_train_acc,mean_test_acc,mean_train_auc,mean_test_auc,param in zip(
                                        mean_train_accuracy,mean_test_accuracy,
                                        mean_train_roc_auc,mean_test_roc_auc,params):
    print("train_acc:{}---test_acc:{}  with:   {}\n" .format(mean_train_acc,mean_test_acc,param),"\n",
         "train_auc:{}---test_auc:{}  with:   {}\n" .format(mean_train_auc,mean_test_auc,param))
    
print("\n grid.best_score_:",grid.best_score_ ,"\n best_params_:",grid.best_params_)
#print(grid_result.cv_results_.keys())

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END ....................C=1000, gamma=scale, kernel=rbf; total time=   6.0s
[CV] END ....................C=1000, gamma=scale, kernel=rbf; total time=   5.9s
[CV] END ....................C=1000, gamma=scale, kernel=rbf; total time=   5.9s
[CV] END ....................C=1000, gamma=scale, kernel=rbf; total time=   5.9s
[CV] END ....................C=1000, gamma=scale, kernel=rbf; total time=   6.0s
[CV] END ....................C=1000, gamma=1e-06, kernel=rbf; total time=   6.3s
[CV] END ....................C=1000, gamma=1e-06, kernel=rbf; total time=   6.1s
[CV] END ....................C=1000, gamma=1e-06, kernel=rbf; total time=   7.1s
[CV] END ....................C=1000, gamma=1e-06, kernel=rbf; total time=   6.2s
[CV] END ....................C=1000, gamma=1e-06, kernel=rbf; total time=   6.0s
[CV] END ....................C=1000, gamma=1e-07, kernel=rbf; total time=   5.9s
[CV] END ....................C=1000, gamma=1e-07,

 - grid.best_score_: 0.7471230987574978 
 - best_params_: {'C': 1000, 'gamma': 1e-06, 'kernel': 'rbf'}

### Test

In [10]:
#y_test-binary-int;y_pred-proba-float
def test_metric(y_test,y_pred):
    y_test=np.array(y_test)
    y_pred=np.array(y_pred)
    
    y_pred_bina = np.around(y_pred,0).astype(int)
    class_acc=accuracy_score(y_test,y_pred_bina)
    
    y_test_int = y_test.astype(int)
    fpr,tpr,threshold=roc_curve(y_test_int,y_pred)#y_test-int(ndarray)
    print(len(fpr))
    roc_auc=auc(fpr,tpr)
    
    print('accuracy {}\t'.format(str(class_acc)), 'AUC:{}\n'.format(roc_auc))

    return np.array(class_acc),np.array(roc_auc)


In [14]:
#SVM default set :probability=False
clf=SVC(random_state=seed,
        kernel="rbf",
        C=1000,
        gamma=0.000001,
        probability=True)

clf.fit(fea_train,y_train)

predictions=[i[1] for i in clf.predict_proba(fea_test)]

accuracy,AUC = test_metric(y_test,predictions)

print('predictions lenth：{}\n'.format(len(predictions)),
    'accuracy {} \n'.format(accuracy),
     'AUC {} \n'.format(AUC))


382
accuracy 0.6714876033057852	 AUC:0.7311501324446724

predictions lenth：968
 accuracy 0.6714876033057852 
 AUC 0.7311501324446724 



In [15]:
AUC_data = np.array([list(y_test),list(predictions)])
print(AUC_data.shape)
AUC_data_df=pd.DataFrame(AUC_data.T,columns=["label","predict"])
AUC_data_df.head()

(2, 968)


,label,predict
0,0.0,0.375449
1,0.0,0.599210
2,0.0,0.536057
3,1.0,0.753688
4,0.0,0.392776


In [17]:
# AUC_data_df.to_csv("auc_data_svc.csv")